In [3]:
import numpy as np
import os


In [4]:
def calc_ranks(dataset, user_count=1000, item_count=1000000, part="train"):
    assert part in ["train", "test"]
    score_path = "{}/data/model_scores/scores_{}.bin".format(dataset, part)
    ranks_path = "{}/data/item_{}_ranks.bin".format(dataset, part)
    log_ranks_path = "{}/data/item_{}_log_ranks.bin".format(dataset, part)
    
    with open(score_path, "rb") as score_file:
        scores = np.fromfile(score_file, dtype="float32")
    assert scores.shape[0] == user_count * item_count
    scores = scores.reshape((item_count, user_count))
    ranks = (-scores).argsort(axis=0).argsort(axis=0)
    ranks = ranks.astype("float32") + 1.0
    ranks.tofile(ranks_path)
    np.log(ranks).tofile(log_ranks_path)


In [5]:
# test
test_scores = np.array([
    [1, 2, 6],
    [4, 3, 5],
    [4, 1, 5],
    [3, 2, 6]
]).astype("float32")
with open("test/data/model_scores/scores_train.bin", "wb") as fout:
    test_scores.tofile(fout)

expected_ranks = np.array([
    [4, 2, 1],
    [1, 1, 3],
    [2, 4, 4],
    [3, 3, 2]
]).astype("float32")

calc_ranks("test", 3, 4)

ranks = np.fromfile("test/data/item_train_ranks.bin", dtype="float32")
ranks = ranks.reshape((4, 3))
assert np.all(ranks == expected_ranks), ranks



In [6]:
calc_ranks("collections", part="test")

In [7]:
calc_ranks("video", part="test")